In [1]:
import pandas as pd
import os
import zipfile

# 1. Ler Arquivos

In [2]:
# Sobe um nível no diretório
os.chdir("..")

In [11]:
# Armazena o novo diretório de trabalho atual
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [9]:
# Caminho dos diretorios
dir_preprocessed_II = os.path.join(diretorio_atual, "data_cvm\preprocessed_II")

In [12]:
# Define o diretório
os.chdir(dir_preprocessed_II)

In [13]:
# Ler dados trimestral
df_trim = pd.read_csv("equatorial_fluxo_caixa_trim.csv")

In [14]:
# Ler dados anual
df_anual = pd.read_csv("equatorial_fluxo_caixa_anual.csv")

In [15]:
df = pd.concat([df_anual, df_trim], ignore_index=True)

# 2. Análise Exploratória

## 2.1 Verficando se há meses duplicados 

In [16]:
teste = df[(df['CD_CONTA'] == '6.01') & (df['ANO'] == 2022)] 

In [17]:
display(teste)

,CNPJ_CIA,DENOM_CIA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,DT_INI_EXERC,TIPO_DEM,ANO,MES
786,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-12-31,6.01,Caixa Líquido Atividades Operacionais,3467507.0,2022-01-01,Demonstração do Fluxo de Caixa (Método Indireto),2022,12
3088,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-03-31,6.01,Caixa Líquido Atividades Operacionais,151864.0,2022-01-01,Demonstração do Fluxo de Caixa (Método Indireto),2022,3
3170,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-06-30,6.01,Caixa Líquido Atividades Operacionais,2075768.0,2022-01-01,Demonstração do Fluxo de Caixa (Método Indireto),2022,6
3241,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-09-30,6.01,Caixa Líquido Atividades Operacionais,2827391.0,2022-01-01,Demonstração do Fluxo de Caixa (Método Indireto),2022,9


# 3. Manipulação dos Dados

## 3.1 Deletando as Colunas

In [18]:
df_droped_column_anual = df_anual.drop(
    columns=['DT_FIM_EXERC','DT_INI_EXERC'])

In [19]:
df_droped_column = df.drop(
    columns=['DT_FIM_EXERC','DT_INI_EXERC'])

## 3.2 Inserindo a Coluna Período

### 3.2.1 Anual

In [20]:
df_droped_column_anual['PERIODO'] = 'ANUAL'

### 3.3.2 Trimestral

In [21]:
# Função para inserir a coluns Período
def insere_periodo_trimestral(df):
    df = df.copy()

    # Dicionário para mapear os meses aos trimestres
    mes_para_trimestre = {3: 1, 6: 2, 9: 3, 12: 4}
    
    # Filtrar apenas os anos e meses relevantes
    anos_relevantes = list(range(2011, 2025))
    meses_relevantes = [3, 6, 9, 12]

    # Aplicar a lógica de período trimestral
    df = df[(df['ANO'].isin(anos_relevantes)) & (df['MES'].isin(meses_relevantes))]
    df['PERIODO'] = df.apply(lambda row: f"{mes_para_trimestre[row['MES']]}T{row['ANO']}", axis=1)
    
    return df

In [22]:
# Inserindo a coluna período
df_droped_column = insere_periodo_trimestral(df_droped_column)

In [24]:
teste = df_droped_column[(df_droped_column['CD_CONTA'] == '6.01') & (df_droped_column['ANO'] == 2023)] 
display(teste)

,CNPJ_CIA,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA,TIPO_DEM,ANO,MES,PERIODO
869,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,6.01,Caixa Líquido Atividades Operacionais,3139179.0,Demonstração do Fluxo de Caixa (Método Indireto),2023,12,4T2023
3324,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,6.01,Caixa Líquido Atividades Operacionais,531985.0,Demonstração do Fluxo de Caixa (Método Indireto),2023,3,1T2023
3407,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,6.01,Caixa Líquido Atividades Operacionais,1977713.0,Demonstração do Fluxo de Caixa (Método Indireto),2023,6,2T2023
3481,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,6.01,Caixa Líquido Atividades Operacionais,2468010.0,Demonstração do Fluxo de Caixa (Método Indireto),2023,9,3T2023


## 3.5 Concatenando Dados Anuais e Trimestrais

In [25]:
df_final = pd.concat([df_droped_column, df_droped_column_anual], ignore_index=True)

## 3.6 Alterando o Nome das Colunas

In [26]:
colunas = ['CNPJ','EMPRESA','CONTA','DESCRIÇÃO','VALOR','DEMONSTRATIVO','ANO', 'MES', 'PERIODO']
df_final.columns = colunas

# 4. Exportando os Dados

In [27]:
# Define o diretório de dados processados
# Caminho dos diretorios
dir_processed_data = os.path.join(diretorio_atual, "data_cvm\processed")
os.chdir(dir_processed_data)

In [28]:
df_final.to_csv('fluxo_de_caixa_completo.csv', index=False)